In [ ]:
import math

def crr_rec(s, k, t, v, r, pc, n):
    if n == 0:
        if pc == 1:  # Call option
            return max(0, s - k)
        elif pc == 0:  # Put option
            return max(0, k - s)
    else:
        h = t / n
        u = math.exp(v * math.sqrt(h))
        d = math.exp(-v * math.sqrt(h))
        R = math.exp(r * h)
        q = (R - d) / (u - d)
        # Recursive calls for up and down moves
        optval = (q * crr_rec(s * u, k, t - h, v, r, pc, n - 1) +
                  (1 - q) * crr_rec(s * d, k, t - h, v, r, pc, n - 1)) / R
        return optval

# Example usage (as in the image):
callopt = crr_rec(50, 52, 0.5, 0.3, 0.03, 1, 10)  # Price a call
putopt = crr_rec(50, 52, 0.5, 0.3, 0.03, 0, 10)   # Price a put

print("Call option value:", callopt)
print("Put option value:", putopt)
print("Call-Put Value:", callopt-putopt)


Call option value: 3.755349885569962
Put option value: 4.981170744929176
Call-Put Value: -1.2258208593592137


In [ ]:
import numpy as np

def crr_div(s, k, t, v, rf, div, pc, ae, n):
    """
    CRR Binomial Option Pricing Model with Proportional Dividends

    Parameters:
    s: Spot price
    k: Strike price
    t: Time to maturity (years)
    v: Volatility (annual)
    rf: Risk-free rate (annual)
    div: Dividend yield (annual)
    pc: Put/Call flag (1=call, 0=put)
    ae: American/European flag (1=American, 0=European)
    n: Number of steps

    Returns:
    Option price
    """
    # Pre-calculate model parameters
    h = t / n
    u = np.exp(v * np.sqrt(h))
    d = 1 / u
    r = np.exp(rf * h)
    y = np.exp(div * h) - 1
    q = (np.exp((rf - div) * h) - d) / (u - d)

    # Initialize trees (using 1-based indexing simulation)
    stktree = np.zeros((n + 2, n + 2))
    divtree = np.zeros((n + 2, n + 2))
    optval = np.zeros((n + 2, n + 2))
    pvdiv = np.zeros((n + 2, n + 2))

    # Initialize first node
    stktree[1, 1] = s
    divtree[1, 1] = 0

    # Build stock price and dividend trees
    for t_step in range(2, n + 2):
        # Top node of current time step
        stktree[1, t_step] = stktree[1, t_step - 1] * u
        divtree[1, t_step] = stktree[1, t_step] * y

        # Subsequent nodes in current time step
        for j in range(2, t_step + 1):
            stktree[j, t_step] = stktree[j - 1, t_step - 1] * d
            divtree[j, t_step] = stktree[j, t_step] * y

    # Terminal payoff calculation
    for j in range(1, n + 2):
        if pc == 1:  # Call option
            optval[j, n + 1] = max(0, stktree[j, n + 1] - k)
        else:  # Put option
            optval[j, n + 1] = max(0, k - stktree[j, n + 1])
        pvdiv[j, n + 1] = divtree[j, n + 1]

    # Backward recursion through the tree
    for t_step in range(n, 0, -1):
        for j in range(1, t_step + 1):
            # Discounted expected value of dividends and option
            pvdiv[j, t_step] = (q * pvdiv[j, t_step + 1] + (1 - q) * pvdiv[j + 1, t_step + 1]) / r
            optval[j, t_step] = (q * optval[j, t_step + 1] + (1 - q) * optval[j + 1, t_step + 1]) / r

            # Add current dividend to PV
            pvdiv[j, t_step] += divtree[j, t_step]

            # American exercise check
            if ae == 1:
                if pc == 1:  # Call
                    exercise_value = stktree[j, t_step] + pvdiv[j, t_step] - k
                else:  # Put
                    exercise_value = k - stktree[j, t_step] + pvdiv[j, t_step]
                optval[j, t_step] = max(optval[j, t_step], exercise_value)

    return optval[1, 1]

# Test with parameters from Octave example
if __name__ == "__main__":
    call_price = crr_div(50, 52, 0.5, 0.3, 0.03, 0.03, 1, 1, 10)
    put_price = crr_div(50, 52, 0.5, 0.3, 0.03, 0.03, 0, 0, 10)

    print(f"American Call Price: {call_price:.4f}")  # Expected: 3.7553
    print(f"European Put Price: {put_price:.4f}")    # Expected: 4.9812


American Call Price: 3.4760
European Put Price: 5.3654


In [ ]:
import numpy as np

def binomial_tree_option(S, X, T, r, sigma, N, option_type='call'):
    """
    Calculate European option price using binomial tree model

    Parameters:
    S (float): Current stock price
    X (float): Strike price
    T (float): Time to expiration (years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of returns
    N (int): Number of time steps
    option_type (str): 'call' or 'put' (default: 'call')

    Returns:
    float: Option price
    """
    delta_t = T / N
    u = np.exp(sigma * np.sqrt(delta_t))
    d = 1 / u
    p = (np.exp(r * delta_t) - d) / (u - d)

    # Initialize asset prices at maturity
    prices = np.zeros(N + 1)
    option_values = np.zeros(N + 1)

    for i in range(N + 1):
        prices[i] = S * (u ** (N - i)) * (d ** i)

    # Calculate option values at maturity
    for i in range(N + 1):
        if option_type == 'call':
            option_values[i] = max(0, prices[i] - X)
        elif option_type == 'put':
            option_values[i] = max(0, X - prices[i])

    # Step back through the tree
    for j in range(N - 1, -1, -1):
        prev_option_values = option_values.copy()
        for i in range(j + 1):
            option_values[i] = np.exp(-r * delta_t) * (
                p * prev_option_values[i] + (1 - p) * prev_option_values[i + 1]
            )

    return option_values[0]

# Example usage
S = 200   # Current stock price
X = 205   # Strike price
T = 2     # Time to expiration in years
r = 0.05  # Risk-free interest rate
sigma = 0.3  # Volatility
N = 3     # Number of steps

call_price = binomial_tree_option(S, X, T, r, sigma, N, 'call')
put_price = binomial_tree_option(S, X, T, r, sigma, N, 'put')

print(f"Call option price: ${call_price:.2f}")
print(f"Put option price: ${put_price:.2f}")


Call option price: $42.48
Put option price: $27.97


In [ ]:
import numpy as np
import scipy.stats as si

def black_scholes(S, X, T, r, sigma):
    """
    Calculate Black-Scholes option prices for both call and put options.

    Parameters:
    S (float) : Current stock price
    X (float) : Strike price
    T (float) : Time to expiration (in years)
    r (float) : Risk-free interest rate
    sigma (float) : Volatility of returns

    Returns:
    tuple: (call_price, put_price)
    """
    # Calculate d1 and d2 parameters
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Calculate call option price
    call_price = (S * si.norm.cdf(d1, 0, 1)) - (X * np.exp(-r * T) * si.norm.cdf(d2, 0, 1))

    # Calculate put option price using put-call parity
    put_price = (X * np.exp(-r * T) * si.norm.cdf(-d2, 0, 1)) - (S * si.norm.cdf(-d1, 0, 1))

    return call_price, put_price

# Substituted values
S = 200    # Current stock price
X = 205    # Strike price
T = 2      # Time to expiration in years
r = 0.05   # Risk-free interest rate
sigma = 0.3  # Volatility of the stock

# Calculate both call and put option prices
call_price, put_price = black_scholes(S, X, T, r, sigma)

print(f"The price of the European call option is: ${call_price:.2f}")
print(f"The price of the European put option is: ${put_price:.2f}")


The price of the European call option is: $40.14
The price of the European put option is: $25.63
